### **Setup**

In [ ]:
# Importing Libraries

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model


### **Model Creation**

In [ ]:
# Training & Testing Paths

trainPath = '/content/dataset/train'
testPath = '/content/dataset/test'


In [ ]:
# Load and Configure VGG16 Model


imageSize = (224, 224)  # Define image size
vgg = VGG16(input_shape=imageSize + (3,), weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(8, activation='softmax')(x)  # 8 classes for tea leaf diseases

model = Model(inputs=vgg.input, outputs=prediction)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
# ImageDataGenerator Configuration for Data Augmentation:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    trainPath,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    testPath,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 706 images belonging to 8 classes.
Found 179 images belonging to 8 classes.


### **Model Training**

In [ ]:
# Training the Model

r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=30
)


Epoch 1/30
23/23 [==============================] - 549s 24s/step - loss: 2.1092 - accuracy: 0.3258 - val_loss: 1.4178 - val_accuracy: 0.4972
Epoch 2/30
23/23 [==============================] - 43s 2s/step - loss: 1.1890 - accuracy: 0.5907 - val_loss: 1.0249 - val_accuracy: 0.6145
Epoch 3/30
23/23 [==============================] - 43s 2s/step - loss: 0.7187 - accuracy: 0.7436 - val_loss: 0.9063 - val_accuracy: 0.6704
Epoch 4/30
23/23 [==============================] - 41s 2s/step - loss: 0.5722 - accuracy: 0.7932 - val_loss: 0.9021 - val_accuracy: 0.6704
Epoch 5/30
23/23 [==============================] - 41s 2s/step - loss: 0.6234 - accuracy: 0.7960 - val_loss: 0.9940 - val_accuracy: 0.6369
Epoch 6/30
23/23 [==============================] - 42s 2s/step - loss: 0.4563 - accuracy: 0.8399 - val_loss: 0.7574 - val_accuracy: 0.6983
Epoch 7/30
23/23 [==============================] - 43s 2s/step - loss: 0.3418 - accuracy: 0.8796 - val_loss: 0.6756 - val_accuracy: 0.7430
Epoch 8/30
23/23 [

In [ ]:
# saving the model

model.save("vgg_16_tea_leaf_disease.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### **Testing The Model**

In [ ]:
# Loading and preparing test image
# Chosen leaf disease image is Antracnose

img_path = "/content/drive/MyDrive/Sem 5 Externship/project/tea_sickness_dataset/test/Anthracnose/IMG_20220503_145703.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)
img_data = preprocess_input(x)


In [ ]:
# Load the pre-trained model
model = load_model('/content/drive/MyDrive/Sem 5 Externship/project/vgg_16_tea_leaf_disease.h5')

# Make a prediction on the image
output = np.argmax(model.predict(img_data), axis=1)

# Mapping the index to class names
index = ['Antracnose', 'algal leaf', 'bird eye spot', 'brown light', 'gray light', 'healthy', 'red leaf spot', 'white spot']
result = index[output[0]]

result